In [8]:
from catboost import CatBoostClassifier
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

In [9]:
PATH = '/Users/tompease/Documents/Coding/fraud_detection'

train = pd.read_csv(f'{PATH}/data/train.csv')
test = pd.read_csv(f'{PATH}/data/test.csv')

features = list(train.columns)
features.remove('Class')
features.remove('id')
features.remove('Time')

target = 'Class'

clfs = []
scores = []

kf = StratifiedKFold(n_splits=10, random_state=0, shuffle=True)

In [15]:

for train_index, val_index in kf.split(train, y=train[target]):
    X_train, X_val = train[features].loc[train_index], train[features].loc[val_index]
    y_train, y_val = train[target][train_index], train[target][val_index]

    clf = CatBoostClassifier(iterations=200)
    clf.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=False)
    
    preds = clf.predict_proba(X_val.values)[:, 1]
    clfs.append(clf)
    scores.append(roc_auc_score(y_val, preds))


    print(f'mean auc across all folds: {np.mean(scores)}')

mean auc across all folds: 0.8090085283561729


In [21]:
test_preds = []

for clf in clfs:
    preds = clf.predict_proba(test[features].values)
    test_preds.append(preds[:, 1])

test_preds = np.stack(test_preds).mean(0)
test_preds

/Users/tompease/opt/miniconda3/envs/ds-env/lib/python3.10/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(
/Users/tompease/opt/miniconda3/envs/ds-env/lib/python3.10/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


array([0.00103726, 0.00104729, 0.00053851, ..., 0.00091494, 0.00037864,
       0.00156739])

In [22]:
submission = pd.DataFrame(data={'id': test.id, 'Class': test_preds})
submission.head()

,id,Class
0,219129,0.001037
1,219130,0.001047
2,219131,0.000539
3,219132,0.001717
4,219133,0.000537


In [ ]:
submission.to_csv('submission2.csv', index=False)